# Init

In [2]:
import logging
import sys
logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
    format='%(asctime)s %(name)s-%(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S')
import os
import numpy as np
from scipy.spatial.distance import squareform
import modules.Compute_Relevance as Compute_Relevance
import matplotlib.pyplot as plt

logger = logging.getLogger("relProp")

# INPUT DATA and INPUT PARAMETERS

In [49]:
points_to_keep = []
# Set number of iterations
n_iter = 10

# Set number of halves to use for relevance estimation
n_halves = 2

# Set parameters of the neural network
hidden_layer_sizes = (100,)


env = "oliver_gpcr" #TODO load these things from config file / environment parameters

if env == "oliver_gpcr":
    home_dir = os.path.expanduser("~/projects/gpcr/mega/Result_Data/beta2-dror/clustering/")
    #data = np.load(home_dir + "frame_distances_CA_inv.npy")
    data = np.load(home_dir + "training_samples_CA_inv.npy")
    clustering = np.loadtxt(home_dir + 'cluster_indices_.txt')
elif env == "marina":
    #home_dir = '/media/mkasimova/Data2/Anton.PRODUCTION_RUN/COMBINED/analysis/CLUSTERING.CHOSEN.CV/NEURAL.NETWORK/marina-relevance-prop/ALL_SC/'
    home_dir = '/media/mkasimova/Data2/calmodulin_from_Annie/'

    # Load the array with collective variables and clusters indices
    data = np.load(home_dir+'frame_i_j_contacts_dt1.npy')
    #clustering = np.loadtxt('/media/mkasimova/Data2/Anton.PRODUCTION_RUN/COMBINED/analysis/CLUSTERING.CHOSEN.CV/NEURAL.NETWORK/marina-relevance-prop/BIAS.CVs/cluster_indices_mean.0.sigma.0.67.2.txt')
    clustering = np.loadtxt(home_dir+'labels.dat')


    # Set points to keep for further analysis (some points should be discarded if clustering is not clean)
    # Set empty if all points should be kept
    #points_to_keep = [[0,1450],\
    #                 [1750,3650],\
    #                 [4150,5900],\
    #                 [6800,-1]]
else:
    raise Exception("No configuration for {}".format(env))

logger.info("Loaded data from %s", env)

2018-10-25 16:25:37 relProp-INFO: Loaded data from oliver_gpcr


# Pre-processing the data

In [50]:
# Vectorize data if it is given in the form of matrices
data = Compute_Relevance.vectorize(data)

# Discard some points if needed
data, clustering = Compute_Relevance.keep_datapoints(data,clustering,points_to_keep)

# Convert inverse distances to contacts using a pre-defined cutoff
#data = Compute_Relevance.convert_to_contact(data)
logger.info("Loaded data of shape %s and %s clusters", data.shape, len(set(clustering)))

2018-10-25 16:25:39 relProp-INFO: Loaded data of shape (857, 40186) and 3 clusters


## Filtering the data

In [42]:
data_init = np.copy(data)

# Contact cutoff based data filtering
contact_filtered_ind, data = Compute_Relevance.filter_by_contact_cutoff(data)

# Kullback-Leibler divergence based data filtering (also works with contacts)
DKL, DKL_filtered_ind, data = Compute_Relevance.filter_by_DKL(data,clustering)
logger.info("Done")

2018-10-25 16:24:02 Compute Relevance-INFO: Number of features before contact cutoff based filtering is 40186
2018-10-25 16:24:02 Compute Relevance-INFO: Number of features after contact cutoff based filtering is 0
2018-10-25 16:24:02 Compute Relevance-INFO: Number of features before DKL based filtering is 0
2018-10-25 16:24:02 Compute Relevance-INFO: Bin size for probability calculation is nan
2018-10-25 16:24:02 Compute Relevance-INFO: Number of features after DKL based filtering is 0
2018-10-25 16:24:02 relProp-INFO: Done


modules/Compute_Relevance.py:101: RuntimeWarning: divide by zero encountered in divide
  data = 1/data
/home/oliverfl/anaconda2/envs/py2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/oliverfl/anaconda2/envs/py2/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/oliverfl/anaconda2/envs/py2/lib/python2.7/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/oliverfl/anaconda2/envs/py2/lib/python2.7/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/oliverfl/anaconda2/envs/py2/lib/python2.7/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# RUN the CODE

In [29]:
# Output relevance[number of clusters, number of features, av/std]
# Output error[n_iter*n_halves]
relevance, error = Compute_Relevance.perform_relevance_propagation(data,\
                                                                   clustering,\
                                                                   hidden_layer_sizes,\
                                                                   n_iter,\
                                                                   n_halves,\
                                                                   scaling=True)


logger.info("Done")

2018-10-25 11:34:59 Compute Relevance-INFO: Performing relevance propagation for the dataset with 12 features and 857 samples
2018-10-25 11:34:59 Compute Relevance-INFO: Number of clusters is 3
2018-10-25 11:34:59 Compute Relevance-INFO: Running iteration 1 ...
2018-10-25 11:34:59 Compute Relevance-INFO: 			Scaling the input dataset ...
2018-10-25 11:34:59 Compute Relevance-INFO: 			Training the neural network ...
2018-10-25 11:34:59 Compute Relevance-INFO: 			Checking for overfit ...
2018-10-25 11:34:59 Compute Relevance-INFO: 			Overfit error is 0.233644859813
2018-10-25 11:34:59 Compute Relevance-INFO: 			Error is less than 5%, therefore computing relevance ...
2018-10-25 11:34:59 Compute Relevance-INFO: 			Rescaling relevance according to min and max in each frame ...
2018-10-25 11:34:59 Compute Relevance-INFO: 			... and averaging it over each cluster
2018-10-25 11:34:59 Compute Relevance-INFO: 			Scaling the input dataset ...
2018-10-25 11:34:59 Compute Relevance-INFO: 			Trainin

2018-10-25 11:35:02 Compute Relevance-INFO: 			Rescaling relevance according to min and max in each frame ...
2018-10-25 11:35:02 Compute Relevance-INFO: 			... and averaging it over each cluster
2018-10-25 11:35:02 Compute Relevance-INFO: 			Scaling the input dataset ...
2018-10-25 11:35:02 Compute Relevance-INFO: 			Training the neural network ...
2018-10-25 11:35:02 Compute Relevance-INFO: 			Checking for overfit ...
2018-10-25 11:35:02 Compute Relevance-INFO: 			Overfit error is 0.0
2018-10-25 11:35:02 Compute Relevance-INFO: 			Error is less than 5%, therefore computing relevance ...
2018-10-25 11:35:02 Compute Relevance-INFO: 			Rescaling relevance according to min and max in each frame ...
2018-10-25 11:35:02 Compute Relevance-INFO: 			... and averaging it over each cluster
2018-10-25 11:35:02 Compute Relevance-INFO: Running iteration 8 ...
2018-10-25 11:35:02 Compute Relevance-INFO: 			Scaling the input dataset ...
2018-10-25 11:35:02 Compute Relevance-INFO: 			Training the neu

# WRITE RESULTS

In [164]:
# Remapping relevance to all input features (if filtering was applied)

number_of_clusters = relevance.shape[0]

try:
    data_init
except NameError:
    pass
else:
    relevance_init = np.zeros((number_of_clusters,data_init.shape[1],2))
    try:
        contact_filtered_ind
        DKL_filtered_ind
    except NameError:
        pass
    else:
        if (len(contact_filtered_ind)>=len(DKL_filtered_ind)):
            ind_1 = contact_filtered_ind
            ind_2 = DKL_filtered_ind
            DKL_init = np.zeros((data_init.shape[1]))
            DKL_init[ind_1] = DKL
            DKL = np.copy(DKL_init)
        else:
            ind_2 = contact_filtered_ind
            ind_1 = DKL_filtered_ind
        relevance_2 = relevance
        relevance_1 = np.zeros((number_of_clusters,len(ind_1),2))
        relevance_1[:,ind_2,:] = relevance_2
        relevance_init[:,ind_1,:] = relevance_1
        relevance = np.copy(relevance_init)
    try:
        contact_filtered_ind
    except NameError:
        pass
    else:
        if relevance.shape[1]!=data_init.shape[1]:
            relevance_init[:,contact_filtered_ind,:] = relevance
            relevance = np.copy(relevance_init)
    try:
        DKL_filtered_ind
    except NameError:
        pass
    else:
        if relevance.shape[1]!=data_init.shape[1]:
            relevance_init[:,DKL_filtered_ind,:] = relevance
            relevance = np.copy(relevance_init)

Compute_Relevance.write_results_input_matrix(relevance,home_dir+'analysis/','results.dat')
logger.info("Done")

2018-10-26 11:42:02 Compute Relevance-INFO: Writing the results ...
2018-10-26 11:42:02 Compute Relevance-INFO: Sigma based filtering of relevances is False
2018-10-26 11:42:03 Compute Relevance-INFO: Done!
2018-10-26 11:42:03 relProp-INFO: Done
